In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Dense
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
print('Tensorflow/Keras: %s' % keras.__version__) # print version

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.model_selection import train_test_split # for splitting data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Tensorflow/Keras: 2.6.0
sklearn: 1.0.2
plotly: 5.7.0
/kaggle/input/iris/Iris.csv
/kaggle/input/iris/database.sqlite


In [63]:
df = pd.read_csv('../input/iris/Iris.csv')

df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [64]:
df["flag1"] = df["Species"].apply(lambda x: 1 if x=="Iris-setosa" else 0)
df["flag2"] = df["Species"].apply(lambda x: 1 if x=="Iris-versicolor" else 0)
df["flag3"] = df["Species"].apply(lambda x: 1 if x=="Iris-virginica" else 0)

df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,flag1,flag2,flag3
0,1,5.1,3.5,1.4,0.2,Iris-setosa,1,0,0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,1,0,0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,1,0,0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,1,0,0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,1,0,0
...,...,...,...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica,0,0,1
146,147,6.3,2.5,5.0,1.9,Iris-virginica,0,0,1
147,148,6.5,3.0,5.2,2.0,Iris-virginica,0,0,1
148,149,6.2,3.4,5.4,2.3,Iris-virginica,0,0,1


In [65]:
X=df.iloc[:,1:5]
y=df.iloc[:,6:9]
# y=df['flag'].values
X,y

(     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
 0              5.1           3.5            1.4           0.2
 1              4.9           3.0            1.4           0.2
 2              4.7           3.2            1.3           0.2
 3              4.6           3.1            1.5           0.2
 4              5.0           3.6            1.4           0.2
 ..             ...           ...            ...           ...
 145            6.7           3.0            5.2           2.3
 146            6.3           2.5            5.0           1.9
 147            6.5           3.0            5.2           2.0
 148            6.2           3.4            5.4           2.3
 149            5.9           3.0            5.1           1.8
 
 [150 rows x 4 columns],
      flag1  flag2  flag3
 0        1      0      0
 1        1      0      0
 2        1      0      0
 3        1      0      0
 4        1      0      0
 ..     ...    ...    ...
 145      0      0      1
 146     

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [67]:
# Specify the structure of a Neural Network
model = Sequential(name="Model_Input") # Model
model.add(Input(shape=(4,), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Dense(4, activation='softplus', name='Hidden-Layer')) # Hidden Layer, softplus(x) = log(exp(x) + 1)
model.add(Dense(3, activation='sigmoid', name='Output-Layer')) # Output Layer, sigmoid(x) = 1 / (1 + exp(-x))


In [68]:
# Compile keras model
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='binary_crossentropy', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['Accuracy', 'Precision', 'Recall'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )

# or loss = Sigmoid Cross-Entropy

In [69]:
X_train,y_train

(     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
 137            6.4           3.1            5.5           1.8
 84             5.4           3.0            4.5           1.5
 27             5.2           3.5            1.5           0.2
 127            6.1           3.0            4.9           1.8
 132            6.4           2.8            5.6           2.2
 ..             ...           ...            ...           ...
 9              4.9           3.1            1.5           0.1
 103            6.3           2.9            5.6           1.8
 67             5.8           2.7            4.1           1.0
 117            7.7           3.8            6.7           2.2
 47             4.6           3.2            1.4           0.2
 
 [120 rows x 4 columns],
      flag1  flag2  flag3
 137      0      0      1
 84       0      1      0
 27       1      0      0
 127      0      0      1
 132      0      0      1
 ..     ...    ...    ...
 9        1      0      0
 103     

In [ ]:
##### Step 5 - Fit keras model on the dataset
model.fit(X_train, # input data
          y_train, # target data
          batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
          epochs=100, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
          verbose='auto', # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
          callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
          validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
          #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
          shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
          class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
          sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
          initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
          steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
          validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
          validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
          validation_freq=3, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
          max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
          workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
          use_multiprocessing=False, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
         )

In [150]:
asss = model.predict(X_train)
df_result = pd.DataFrame(asss)

In [151]:
df_result

,0,1,2
0,9.045245e-07,0.306279,0.835627
1,4.063128e-05,0.291594,0.453858
2,9.997086e-01,0.144340,0.000001
3,4.256483e-06,0.299380,0.708095
4,4.463875e-08,0.344997,0.981074
...,...,...,...
115,9.994905e-01,0.164287,0.000002
116,4.046785e-07,0.326846,0.910295
117,2.606750e-03,0.270268,0.026345
118,2.387072e-08,0.280966,0.948446


In [152]:
for x in range(120):
    df_result.loc[x,df_result[x:x+1].idxmax(axis="columns")]=1
    
df_result = (df_result == 1.0).astype(int)
df_result,y_train

(     0  1  2
 0    0  0  1
 1    0  0  1
 2    1  0  0
 3    0  0  1
 4    0  0  1
 ..  .. .. ..
 115  1  0  0
 116  0  0  1
 117  0  1  0
 118  0  0  1
 119  1  0  0
 
 [120 rows x 3 columns],
      flag1  flag2  flag3
 137      0      0      1
 84       0      1      0
 27       1      0      0
 127      0      0      1
 132      0      0      1
 ..     ...    ...    ...
 9        1      0      0
 103      0      0      1
 67       0      1      0
 117      0      0      1
 47       1      0      0
 
 [120 rows x 3 columns])

In [153]:
print(classification_report(y_train, df_result))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      0.84      0.91        37
           2       0.88      1.00      0.94        44

   micro avg       0.95      0.95      0.95       120
   macro avg       0.96      0.95      0.95       120
weighted avg       0.96      0.95      0.95       120
 samples avg       0.95      0.95      0.95       120



In [154]:
model.summary() 

Model: "Model_Input"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Hidden-Layer (Dense)         (None, 4)                 20        
_________________________________________________________________
Output-Layer (Dense)         (None, 3)                 15        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [158]:
attt = model.predict(X_test)
kkkk = pd.DataFrame(attt)

for y in range(30):
    kkkk.loc[y,kkkk[y:y+1].idxmax(axis="columns")]=1
    
kkkk = (df_result2 == 1).astype(int)
kkkk,y_test

print(classification_report(y_test, df_result2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      0.92      0.96        13
           2       0.86      1.00      0.92         6

   micro avg       0.97      0.97      0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30
 samples avg       0.97      0.97      0.97        30



In [72]:
df_result.to_csv('df_result.csv', index=False)